In [6]:
!pip install torch==1.0.0 torchvision==0.2.1
!pip install Pillow==6.2.2

In [16]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.0.0
True


In [24]:
############################################## model.py ####################################################
import math
import torch
from torch import nn

class Generator(nn.Module):
    def __init__(self, upscale_factor: int = 4) -> None:
        r"""
        Args:
            upscale_factor (int): How many times to upscale the picture. (Default: 4)
        """
        super(Generator, self).__init__()
        # Calculating the number of subpixel convolution layers.
        num_subpixel_convolution_layers = int(math.log(upscale_factor, 2))

        # First layer.
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=9, stride=1, padding=4),
            nn.PReLU()
        )

        # 16 Residual blocks.
        trunk = []
        for _ in range(2):
            trunk.append(ResidualBlock(channels=8))
        self.trunk = nn.Sequential(*trunk)

        # Second conv layer post residual blocks.
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 8, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(8)
        )

        # 2 Sub-pixel convolution layers.
        subpixel_conv_layers = []
        for _ in range(num_subpixel_convolution_layers):
            subpixel_conv_layers.append(SubpixelConvolutionLayer(8))
        self.subpixel_conv = nn.Sequential(*subpixel_conv_layers)

        # Final output layer.
        self.conv3 = nn.Conv2d(8, 3, kernel_size=9, stride=1, padding=4)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        conv1 = self.conv1(x)
        trunk = self.trunk(conv1)
        conv2 = self.conv2(trunk)
        out = torch.add(conv1, conv2)
        out = self.subpixel_conv(out)
        out = self.conv3(out)

        return out

class Discriminator(nn.Module):
    def __init__(self, image_size: int = 96) -> None:
        super(Discriminator, self).__init__()

        feature_map_size = int(image_size // 16)

        self.features = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=2, padding=1, bias=False),  # input is (3) x 96 x 96
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1, bias=False), # input size. (8) x 48 x 48
            nn.BatchNorm2d(16),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1, bias=False), # input size. (16) x 24 x 24
            nn.BatchNorm2d(32),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),

            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1, bias=False), # input size. (32) x 12 x 12
            nn.BatchNorm2d(64),
            nn.LeakyReLU(negative_slope=0.2, inplace=True), # output size. (64) x 6 x 6
        )

        self.classifier = nn.Sequential(
            nn.Linear(64 * feature_map_size * feature_map_size, 1024),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.features(x)
        out = torch.flatten(out, 1)
        out = self.classifier(out)

        return out

class SubpixelConvolutionLayer(nn.Module):
    def __init__(self, channels: int = 64) -> None:
        r"""
        Args:
            channels (int): Number of channels in the input image. (Default: 64)
        """
        super(SubpixelConvolutionLayer, self).__init__()
        self.conv = nn.Conv2d(channels, channels * 4, kernel_size=3, stride=1, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor=2)
        self.prelu = nn.PReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.conv(x)
        out = self.pixel_shuffle(out)
        out = self.prelu(out)

        return out

class ResidualBlock(nn.Module):
    def __init__(self, channels: int = 64) -> None:
        r"""
        Args:
            channels (int): Number of channels in the input image. (Default: 64)
        """
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)
        self.prelu = nn.PReLU()
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.prelu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        out = out + x

        return out

############################################# loss.py ###########################################################
from torchvision.models.vgg import vgg16


class GeneratorLoss(nn.Module):
    def __init__(self):
        super(GeneratorLoss, self).__init__()
        vgg = vgg16(pretrained=True)
        loss_network = nn.Sequential(*list(vgg.features)[:31]).eval()
        for param in loss_network.parameters():
            param.requires_grad = False
        self.loss_network = loss_network
        self.mse_loss = nn.MSELoss()
        self.tv_loss = TVLoss()

    def forward(self, out_labels, out_images, target_images):
        # Adversarial Loss
        adversarial_loss = torch.mean(1 - out_labels)
        # Perception Loss
        perception_loss = self.mse_loss(self.loss_network(out_images), self.loss_network(target_images))
        # Image Loss
        image_loss = self.mse_loss(out_images, target_images)
        # TV Loss
        tv_loss = self.tv_loss(out_images)
        return image_loss + 0.001 * adversarial_loss + 0.006 * perception_loss + 2e-8 * tv_loss


class TVLoss(nn.Module):
    def __init__(self, tv_loss_weight=1):
        super(TVLoss, self).__init__()
        self.tv_loss_weight = tv_loss_weight

    def forward(self, x):
        batch_size = x.size()[0]
        h_x = x.size()[2]
        w_x = x.size()[3]
        count_h = self.tensor_size(x[:, :, 1:, :])
        count_w = self.tensor_size(x[:, :, :, 1:])
        h_tv = torch.pow((x[:, :, 1:, :] - x[:, :, :h_x - 1, :]), 2).sum()
        w_tv = torch.pow((x[:, :, :, 1:] - x[:, :, :, :w_x - 1]), 2).sum()
        return self.tv_loss_weight * 2 * (h_tv / count_h + w_tv / count_w) / batch_size

    @staticmethod
    def tensor_size(t):
        return t.size()[1] * t.size()[2] * t.size()[3]

############################################# data_utils ########################################################
from os import listdir
from os.path import join

from PIL import Image
from torch.utils.data.dataset import Dataset
from torchvision.transforms import Compose, RandomCrop, ToTensor, ToPILImage, CenterCrop, Resize


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG'])


def calculate_valid_crop_size(crop_size, upscale_factor):
    return crop_size - (crop_size % upscale_factor)


def train_hr_transform(crop_size):
    return Compose([
        RandomCrop(crop_size),
        ToTensor(),
    ])


def train_lr_transform(crop_size, upscale_factor):
    return Compose([
        ToPILImage(),
        Resize(crop_size // upscale_factor, interpolation=Image.BICUBIC),
        ToTensor()
    ])


def display_transform():
    return Compose([
        ToPILImage(),
        Resize(400),
        CenterCrop(400),
        ToTensor()
    ])


class TrainDatasetFromFolder(Dataset):
    def __init__(self, dataset_dir, crop_size, upscale_factor):
        super(TrainDatasetFromFolder, self).__init__()
        self.image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir) if is_image_file(x)]
        crop_size = calculate_valid_crop_size(crop_size, upscale_factor)
        self.hr_transform = train_hr_transform(crop_size)
        self.lr_transform = train_lr_transform(crop_size, upscale_factor)

    def __getitem__(self, index):
        hr_image = self.hr_transform(Image.open(self.image_filenames[index]))
        lr_image = self.lr_transform(hr_image)
        return lr_image, hr_image

    def __len__(self):
        return len(self.image_filenames)

class Get_BlurHR_HR_FromPath(Dataset):
    def __init__(self, BlurHR_dataset_dir, HR_dataset_dir):
        super(Get_BlurHR_HR_FromPath, self).__init__()
        self.BlurHR_image_filenames = [join(BlurHR_dataset_dir, y) for y in listdir(BlurHR_dataset_dir) if is_image_file(y)]
        self.HR_image_filenames = [join(HR_dataset_dir, x) for x in listdir(HR_dataset_dir) if is_image_file(x)]

    def __getitem__(self, index):
        BlurHR_image = Image.open(self.BlurHR_image_filenames[index])
        HR_image = Image.open(self.HR_image_filenames[index])
        return ToTensor()(BlurHR_image), ToTensor()(HR_image)

    def __len__(self):
        return len(self.BlurHR_image_filenames)


class ValDatasetFromFolder(Dataset):
    def __init__(self, dataset_dir, upscale_factor):
        super(ValDatasetFromFolder, self).__init__()
        self.upscale_factor = upscale_factor
        self.image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir) if is_image_file(x)]

    def __getitem__(self, index):
        hr_image = Image.open(self.image_filenames[index])
        w, h = hr_image.size
        crop_size = calculate_valid_crop_size(min(w, h), self.upscale_factor)
        lr_scale = Resize(crop_size // self.upscale_factor, interpolation=Image.BICUBIC)
        hr_scale = Resize(crop_size, interpolation=Image.BICUBIC)
        hr_image = CenterCrop(crop_size)(hr_image)
        lr_image = lr_scale(hr_image)
        hr_restore_img = hr_scale(lr_image)
        return ToTensor()(lr_image), ToTensor()(hr_restore_img), ToTensor()(hr_image)

    def __len__(self):
        return len(self.image_filenames)


class TestDatasetFromFolder(Dataset):
    def __init__(self, dataset_dir, upscale_factor):
        super(TestDatasetFromFolder, self).__init__()
        self.lr_path = dataset_dir + '/SRF_' + str(upscale_factor) + '/data/'
        self.hr_path = dataset_dir + '/SRF_' + str(upscale_factor) + '/target/'
        self.upscale_factor = upscale_factor
        self.lr_filenames = [join(self.lr_path, x) for x in listdir(self.lr_path) if is_image_file(x)]
        self.hr_filenames = [join(self.hr_path, x) for x in listdir(self.hr_path) if is_image_file(x)]

    def __getitem__(self, index):
        image_name = self.lr_filenames[index].split('/')[-1]
        lr_image = Image.open(self.lr_filenames[index])
        w, h = lr_image.size
        hr_image = Image.open(self.hr_filenames[index])
        hr_scale = Resize((self.upscale_factor * h, self.upscale_factor * w), interpolation=Image.BICUBIC)
        hr_restore_img = hr_scale(lr_image)
        return image_name, ToTensor()(lr_image), ToTensor()(hr_restore_img), ToTensor()(hr_image)

    def __len__(self):
        return len(self.lr_filenames)



############################################# pytorch_ssim ######################################################
# fixed bugs
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from math import exp

def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()

def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size))
    return window

def _ssim(img1, img2, window, window_size, channel, size_average = True):
    window = window.to(device)
    mu1 = F.conv2d(img1, window, padding = window_size//2, groups = channel)
    mu2 = F.conv2d(img2, window, padding = window_size//2, groups = channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1*mu2

    sigma1_sq = F.conv2d(img1*img1, window, padding = window_size//2, groups = channel) - mu1_sq
    sigma2_sq = F.conv2d(img2*img2, window, padding = window_size//2, groups = channel) - mu2_sq
    sigma12 = F.conv2d(img1*img2, window, padding = window_size//2, groups = channel) - mu1_mu2

    C1 = 0.01**2
    C2 = 0.03**2

    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)

class SSIM(torch.nn.Module):
    def __init__(self, window_size = 11, size_average = True):
        super(SSIM, self).__init__()
        self.window_size = window_size
        self.size_average = size_average
        self.channel = 1
        self.window = create_window(window_size, self.channel)

    def forward(self, img1, img2):
        (_, channel, _, _) = img1.size()

        if channel == self.channel:
            window = self.window
        else:
            window = create_window(self.window_size, channel)
            self.window = window
            self.channel = channel

        return _ssim(img1, img2, window, self.window_size, channel, self.size_average)

def ssim(img1, img2, window_size = 11, size_average = True):
    (_, channel, _, _) = img1.size()
    window = create_window(window_size, channel)
    return _ssim(img1, img2, window, window_size, channel, size_average)



############################################# train.py ##########################################################
import argparse
import os
from math import log10

import pandas as pd
import torch.optim as optim
import torch.utils.data
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm


# parser = argparse.ArgumentParser(description='Train Super Resolution Models')
# parser.add_argument('--crop_size', default=512, type=int, help='training images crop size')
# parser.add_argument('--upscale_factor', default=4, type=int, choices=[2, 4, 8],
#                     help='super resolution upscale factor')
# parser.add_argument('--num_epochs', default=100, type=int, help='train epoch number')


if __name__ == '__main__':
    
    CROP_SIZE = 96
    UPSCALE_FACTOR = 4
    NUM_EPOCHS = 300
    learning_rate = 1e-3
    lr_step_gamma = 0.8
    step_size = 50
    batch_size = 25
    
    train_set = TrainDatasetFromFolder('/content/drive/MyDrive/CS17 project/data/HR/', crop_size=CROP_SIZE, upscale_factor=UPSCALE_FACTOR)
    # train_set = Get_BlurHR_HR_FromPath('/content/drive/My Drive/data/train_BlurHR/', '/content/drive/My Drive/data/train_HR/')
    val_set = ValDatasetFromFolder('/content/drive/MyDrive/CS17 project/data/HR/', upscale_factor=UPSCALE_FACTOR)
    # val_set = Get_BlurHR_HR_FromPath('/content/drive/My Drive/data/train_BlurHR/', '/content/drive/My Drive/data/train_HR/')
    train_loader = DataLoader(dataset=train_set, num_workers=4, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(dataset=val_set, num_workers=4, batch_size=batch_size, shuffle=False)
    
    netG = Generator()
    print('# generator parameters:', sum(param.numel() for param in netG.parameters()))
    netD = Discriminator()
    print('# discriminator parameters:', sum(param.numel() for param in netD.parameters()))
    
    generator_criterion = GeneratorLoss()
    
    if torch.cuda.is_available():
        netG.cuda()
        netD.cuda()
        generator_criterion.cuda()
    
    optimizerG = optim.Adam(netG.parameters(), lr = learning_rate)
    optimizerD = optim.Adam(netD.parameters(), lr = learning_rate)
    schedulerG = optim.lr_scheduler.StepLR(optimizerG, step_size=step_size, gamma=lr_step_gamma)
    schedulerD = optim.lr_scheduler.StepLR(optimizerD, step_size=step_size, gamma=lr_step_gamma)
    
    results = {'d_loss': [], 'g_loss': [], 'd_score': [], 'g_score': [], 'psnr': [], 'ssim': []}
    
    for epoch in range(1, NUM_EPOCHS + 1):
        train_bar = tqdm(train_loader)
        running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 'd_score': 0, 'g_score': 0}
    
        torch.autograd.set_detect_anomaly(True)

        netG.train()
        netD.train()
        for data, target in train_bar:
            g_update_first = True
            batch_size = data.size(0)
            running_results['batch_sizes'] += batch_size
    
            ############################
            # (1) Update D network: maximize D(x)-1-D(G(z)) equals to minimize 1 + D(G(z)) - D(x)
            ###########################
            real_img = Variable(target)
            if torch.cuda.is_available():
                real_img = real_img.cuda()
            z = Variable(data)
            if torch.cuda.is_available():
                z = z.cuda()
            fake_img = netG(z)
    
            netD.zero_grad()
            real_out = netD(real_img).mean() # D(x)
            fake_out = netD(fake_img).mean() # D(G(z))
            d_loss = 1 - real_out + fake_out
            d_loss.backward(retain_graph=True)
            optimizerD.step()
    
            ############################
            # (2) Update G network: minimize 1-D(G(z)) + Perception Loss + Image Loss + TV Loss
            ###########################
            netG.zero_grad()
            g_loss = generator_criterion(fake_out, fake_img, real_img)
            g_loss.backward()
            
            fake_img = netG(z)
            fake_out = netD(fake_img).mean()
            
            optimizerG.step()

            # loss for current batch before optimization 
            running_results['g_loss'] += g_loss.item() * batch_size
            running_results['d_loss'] += d_loss.item() * batch_size
            running_results['d_score'] += real_out.item() * batch_size
            running_results['g_score'] += fake_out.item() * batch_size
    
            train_bar.set_description(desc='[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f' % (
                epoch, NUM_EPOCHS, running_results['d_loss'] / running_results['batch_sizes'],
                running_results['g_loss'] / running_results['batch_sizes'],
                running_results['d_score'] / running_results['batch_sizes'],
                running_results['g_score'] / running_results['batch_sizes']))
            
        schedulerG.step()
        schedulerD.step()

        netG.eval()
        out_path = '/content/drive/My Drive/data/training_results/SRF_' + str(UPSCALE_FACTOR) + '/'
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        
        if epoch % 10 == 0 or epoch == 1:
          with torch.no_grad():
              val_bar = tqdm(val_loader)
              valing_results = {'mse': 0, 'ssims': 0, 'psnr': 0, 'ssim': 0, 'batch_sizes': 0}
              val_images = []
              for val_lr, val_hr_restore, val_hr in val_bar:
                  batch_size = val_lr.size(0)
                  valing_results['batch_sizes'] += batch_size
                  lr = val_lr
                  hr = val_hr
                  if torch.cuda.is_available():
                      lr = lr.cuda()
                      hr = hr.cuda()
                  sr = netG(lr)
                  sr = sr.view(-1, 3, 600, 600) 
          
                  batch_mse = ((sr - hr) ** 2).data.mean()
                  valing_results['mse'] += batch_mse * batch_size
                  batch_ssim = ssim(sr, hr).item()
                  valing_results['ssims'] += batch_ssim * batch_size
                  if hr.max() == 0:
                    valing_results['psnr'] = 0
                  else:
                    valing_results['psnr'] = 10 * log10((hr.max()**2) / (valing_results['mse'] / valing_results['batch_sizes']))
                  valing_results['ssim'] = valing_results['ssims'] / valing_results['batch_sizes']
                  val_bar.set_description(
                      desc='[converting LR images to SR images] PSNR: %.4f dB SSIM: %.4f' % (
                          valing_results['psnr'], valing_results['ssim']))

              # for val_blurhr, val_hr in val_bar:

              #     batch_size = val_blurhr.size(0)
              #     valing_results['batch_sizes'] += batch_size
              #     blurhr = ToTensor()(val_blurhr)
              #     hr = ToTensor()(val_hr)
              #     if torch.cuda.is_available():
              #         blurhr = blurhr.cuda()
              #         hr = hr.cuda()
              #     refined_blurhr = netG(blurhr)
          
              #     batch_mse = ((refined_blurhr - hr) ** 2).data.mean()
              #     valing_results['mse'] += batch_mse * batch_size
              #     batch_ssim = ssim(refined_blurhr, hr).item()
              #     valing_results['ssims'] += batch_ssim * batch_size
              #     if hr.max() == 0:
              #       valing_results['psnr'] = 0
              #     else:
              #       valing_results['psnr'] = 10 * log10((hr.max()**2) / (valing_results['mse'] / valing_results['batch_sizes']))
              #     valing_results['ssim'] = valing_results['ssims'] / valing_results['batch_sizes']
              #     val_bar.set_description(
              #         desc='[Refining BlurHR images to HR images] PSNR: %.4f dB SSIM: %.4f' % (
              #             valing_results['psnr'], valing_results['ssim']))
    
          # save model parameters
          torch.save(netG.state_dict(), '/content/drive/MyDrive/CS17 project/data/epochs/netG_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch))
          # torch.save(netD.state_dict(), '/content/drive/MyDrive/CS17 project/data/epochs/netD_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch))
          # save loss\scores\psnr\ssim
          results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
          results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
          results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
          results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])
          results['psnr'].append(valing_results['psnr'])
          results['ssim'].append(valing_results['ssim'])
    
        if epoch % 10 == 0 or epoch == 1:
            out_path = '/content/drive/MyDrive/CS17 project/data/statistics/'
            data_frame = pd.DataFrame(
                data={'Loss_D': results['d_loss'], 'Loss_G': results['g_loss'], 'Score_D': results['d_score'],
                      'Score_G': results['g_score'], 'PSNR': results['psnr'], 'SSIM': results['ssim']})
            data_frame.to_csv(out_path + 'srf_' + str(UPSCALE_FACTOR) + '_train_results.csv', index_label='Epoch')

# generator parameters: 11536
# discriminator parameters: 2385977


Streaming output truncated to the last 5000 lines.

[233/300] Loss_D: 1.0000 Loss_G: 0.0027 D(x): 1.0000 D(G(z)): 1.0000:  38%|███▊      | 6/16 [00:05<00:08,  1.21it/s]

[233/300] Loss_D: 1.0000 Loss_G: 0.0028 D(x): 1.0000 D(G(z)): 1.0000:  38%|███▊      | 6/16 [00:05<00:08,  1.21it/s]

[233/300] Loss_D: 1.0000 Loss_G: 0.0028 D(x): 1.0000 D(G(z)): 1.0000:  44%|████▍     | 7/16 [00:05<00:06,  1.35it/s]

[233/300] Loss_D: 1.0000 Loss_G: 0.0028 D(x): 1.0000 D(G(z)): 1.0000:  44%|████▍     | 7/16 [00:06<00:06,  1.35it/s]

[233/300] Loss_D: 1.0000 Loss_G: 0.0028 D(x): 1.0000 D(G(z)): 1.0000:  50%|█████     | 8/16 [00:06<00:05,  1.47it/s]

[233/300] Loss_D: 1.0000 Loss_G: 0.0030 D(x): 1.0000 D(G(z)): 1.0000:  50%|█████     | 8/16 [00:06<00:05,  1.47it/s]

[233/300] Loss_D: 1.0000 Loss_G: 0.0030 D(x): 1.0000 D(G(z)): 1.0000:  56%|█████▋    | 9/16 [00:06<00:04,  1.62it/s]

[233/300] Loss_D: 1.0000 Loss_G: 0.0029 D(x): 1.0000 D(G(z)): 1.0000:  56%|█████▋    | 9/16 [00:06<00:04,  1.62it/s]

[233

In [25]:
!python3 -m pip install SSIM-PIL
from SSIM_PIL import compare_ssim

IMAGE_NAME = '/content/drive/MyDrive/CS17 project/data/HR/VikingVillage_600x600_192.png'
MODEL_NAME = '/content/drive/MyDrive/CS17 project/data/epochs/netG_epoch_4_100.pth'
netG.load_state_dict(torch.load(MODEL_NAME))

HR_image = Image.open(IMAGE_NAME)
LR_image = HR_image.resize((150,150), Image.BICUBIC)
LR_image = Variable(ToTensor()(LR_image), volatile=True).unsqueeze(0)

SR_image = netG(LR_image.cuda())
SR_image = SR_image.view(-1, 3, 600, 600) 
SR_image = ToPILImage()(SR_image[0].data.cpu())

ssim_score = compare_ssim(SR_image,HR_image)
print(ssim_score)

SR_image.save("/content/drive/MyDrive/CS17 project/data/test/ASLG_4.png")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.


No module named 'pyopencl'
0.7289226829996849
